In [1]:
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

/home/adam/miniconda/envs/gpt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Bigram model

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O shakespear.txt

--2023-02-11 23:39:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘shakespear.txt’

shakespear.txt      100%[===================>]   1,06M  3,51MB/s    in 0,3s    

2023-02-11 23:39:58 (3,51 MB/s) - ‘shakespear.txt’ saved [1115394/1115394]



In [2]:
with open('shakespear.txt', 'r', encoding='utf-8') as fr:
    text = fr.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocab size: {vocab_size}")
print("".join(chars))

Vocab size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos  = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
n = int(0.9*len(data))
train_data, valid_data = data[:n], data[n:]
print(train_data.shape, valid_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context, target)
    

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(0, len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
    

In [11]:
xb, yb = get_batch('train')
xb, yb

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [12]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(context, target)

tensor([24]) tensor(43)
tensor([24, 43]) tensor(58)
tensor([24, 43, 58]) tensor(5)
tensor([24, 43, 58,  5]) tensor(57)
tensor([24, 43, 58,  5, 57]) tensor(1)
tensor([24, 43, 58,  5, 57,  1]) tensor(46)
tensor([24, 43, 58,  5, 57,  1, 46]) tensor(43)
tensor([24, 43, 58,  5, 57,  1, 46, 43]) tensor(39)
tensor([44]) tensor(53)
tensor([44, 53]) tensor(56)
tensor([44, 53, 56]) tensor(1)
tensor([44, 53, 56,  1]) tensor(58)
tensor([44, 53, 56,  1, 58]) tensor(46)
tensor([44, 53, 56,  1, 58, 46]) tensor(39)
tensor([44, 53, 56,  1, 58, 46, 39]) tensor(58)
tensor([44, 53, 56,  1, 58, 46, 39, 58]) tensor(1)
tensor([52]) tensor(58)
tensor([52, 58]) tensor(1)
tensor([52, 58,  1]) tensor(58)
tensor([52, 58,  1, 58]) tensor(46)
tensor([52, 58,  1, 58, 46]) tensor(39)
tensor([52, 58,  1, 58, 46, 39]) tensor(58)
tensor([52, 58,  1, 58, 46, 39, 58]) tensor(1)
tensor([52, 58,  1, 58, 46, 39, 58,  1]) tensor(46)
tensor([25]) tensor(17)
tensor([25, 17]) tensor(27)
tensor([25, 17, 27]) tensor(10)
tensor([25

In [71]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # Batch Timestamp Channels
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:,-1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss, -np.log(1/vocab_size))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>) 4.174387269895637


In [72]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [ ]:
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [87]:
for step in range(10000):
    xb, yb  = get_batch('train')
    _, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.4007861614227295


In [91]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 100)[0].tolist()))


ge d m pomst' tontean uerend thees:
Anlldonghe. d thabejethard apte ifisouthodibup figucend d here a


### Self attention

In [6]:
# version 1
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn((B, T, C))

xbow = torch.randn((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        # takes just context from curent and previous positions
        xbow[b,t] = torch.mean(xprev, axis=0)

In [7]:
x[0][:3], xbow[0][:3]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199]]))

In [8]:
# version 2
wei = torch.tril(torch.ones((T,T)))
wei  = wei / wei.sum(dim=1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [9]:
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [10]:
# version 3
tril = torch.tril(torch.ones((T, T)))
# this should be trainable in the future vertsion
wei  = torch.zeros((T, T))
wei = F.softmax(wei.masked_fill(tril == 0, -float('inf')), dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [11]:
torch.allclose(wei @ x, xbow)

True

In [12]:
# self attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
head_size = 16

x = torch.randn((B, T, C))
# in case of cross attention key & value come from encoder part
key = nn.Linear(C, head_size, bias=False) 
value = nn.Linear(C, head_size, bias=False)

query = nn.Linear(C, head_size, bias=False)

k = key(x) # B T head_size
q = query(x) # B T head_size
v = value(x)
wei = q @ k.transpose(-2, -1) # B T head_size @ B head_size T -> B T T

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # difference between autoencoder and generator
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [15]:
wei.shape, v.shape

(torch.Size([4, 8, 8]), torch.Size([4, 8, 16]))

In [16]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [17]:
out.shape

torch.Size([4, 8, 16])

In [22]:
# Note on variance
q = torch.randn(B, T, head_size)
k = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)
print(q.var(), k.var(), wei.var(), head_size)

q = torch.randn(B, T, head_size)
k = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
print(q.var(), k.var(), wei.var(), head_size, head_size**-0.5)

tensor(1.0832) tensor(0.8747) tensor(18.0547) 16
tensor(1.0283) tensor(1.0969) tensor(1.1257) 16 0.25


In [28]:
# In case distribution is sharp, softmax is peaky and takes extremes
F.softmax(torch.tensor([[-2,2,5]], dtype=torch.float)), F.softmax(torch.tensor([[-2,2,5]], dtype=torch.float) * 5)

/tmp/ipykernel_237125/3072177423.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.tensor([[-2,2,5]], dtype=torch.float)), F.softmax(torch.tensor([[-2,2,5]], dtype=torch.float) * 5)


(tensor([[8.6788e-04, 4.7385e-02, 9.5175e-01]]),
 tensor([[6.3051e-16, 3.0590e-07, 1.0000e+00]]))

### Batch nomr 1d

In [12]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    def __call__(self, x):
        xmean = x.mean(dim=1, keepdim=True)
        xvar = x.var(dim=1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        return self.gamma * xhat + self.beta
    
    def parameters(self):
        return [self.gamam, self.beta]

In [16]:
torch.manual_seed(1337)
bn = BatchNorm1d(100)

x = torch.randn(32, 100)
x = bn(x)

In [18]:
x[0, :].mean(), x[0, :].std(), ((x[1, :] - x[1, :].mean())/x[1, :].std()).mean(),  ((x[1, :] - x[1, :].mean())/x[1, :].std()).std()

(tensor(-9.5367e-09), tensor(1.0000), tensor(9.5367e-09), tensor(1.))

tensor([[-0.1109],
        [ 0.0118],
        [-0.0243],
        [ 0.0093],
        [-0.1960],
        [ 0.1990],
        [-0.1238],
        [ 0.0915],
        [-0.0573],
        [-0.1076],
        [-0.0401],
        [-0.0425],
        [ 0.1250],
        [-0.1696],
        [ 0.0554],
        [-0.0430],
        [-0.0311],
        [-0.0686],
        [-0.0969],
        [ 0.0574],
        [-0.0121],
        [-0.0852],
        [-0.0028],
        [ 0.1250],
        [-0.0149],
        [-0.0954],
        [ 0.0832],
        [-0.0766],
        [ 0.1630],
        [-0.0633],
        [-0.1054],
        [-0.0549]])